# AOC 2025 Day 9

In [1]:
@file:DependsOn("com.toldoven.aoc:aoc-kotlin-notebook:1.1.2")

In [2]:
import com.toldoven.aoc.notebook.AocClient

val aocClient = AocClient.fromEnv().interactiveDay(2025, 9)
val input = aocClient.input()
aocClient.viewPartOne()

--- Day 9: Movie Theater --- 
 You slide down the firepole in the corner of the playground and land in the North Pole base movie theater! 
 The movie theater has a big tile floor with an interesting pattern. Elves here are redecorating the theater by switching out some of the square tiles in the big grid they form. Some of the tiles are red ; the Elves would like to find the largest rectangle that uses red tiles for two of its opposite corners. They even have a list of where the red tiles are located in the grid (your puzzle input). 
 For example: 
 7,1
11,1
11,7
9,7
9,5
2,5
2,3
7,3
 
 Showing red tiles as # and other tiles as . , the above arrangement of red tiles would look like this: 
 ..............
.......#...#..
..............
..#....#......
..............
..#......#....
..............
.........#.#..
..............
 
 You can choose any two red tiles as the opposite corners of your rectangle; your goal is to find the largest rectangle possible. 
 For example, you could make a rectangle (shown as O ) with an area of 24 between 2,5 and 9,7 : 
 ..............
.......#...#..
..............
..#....#......
..............
.. O OOOOOOO....
..OOOOOOOO....
..OOOOOOO O .#..
..............
 
 Or, you could make a rectangle with area 35 between 7,1 and 11,7 : 
 ..............
....... O OOOO..
.......OOOOO..
..#....OOOOO..
.......OOOOO..
..#....OOOOO..
.......OOOOO..
.......OOOO O ..
..............
 
 You could even make a thin rectangle with an area of only 6 between 7,3 and 2,3 : 
 ..............
.......#...#..
..............
.. O OOOO O ......
..............
..#......#....
..............
.........#.#..
..............
 
 Ultimately, the largest rectangle you can make in this example has area 50 . One way to do this is between 2,5 and 11,1 : 
 ..............
..OOOOOOOOO O ..
..OOOOOOOOOO..
..OOOOOOOOOO..
..OOOOOOOOOO..
.. O OOOOOOOOO..
..............
.........#.#..
..............
 
 Using two red tiles as opposite corners, what is the largest area of any rectangle you can make?

In [4]:
data class Tile(val x: Long, val y: Long)

data class Rectangle(val topLeft: Tile, val bottomRight: Tile) {
    fun size(): Long {
        val width = abs(bottomRight.x - topLeft.x) + 1
        val height = abs(bottomRight.y - topLeft.y) + 1
        return width * height
    }
}

val tiles = input.split("\n").map { line ->
    val (x, y) = line.split(",").map { it.toLong() }
    Tile(x, y)
}

// find all possible rectangles
val rectangles = tiles.flatMap { tile ->
    tiles.map { otherTile ->
        Rectangle(tile, otherTile)
    }
}
    .filter { it.topLeft != it.bottomRight }

val answerPartOne = rectangles.maxByOrNull { it.size() }?.size() ?: 0

aocClient.submitPartOne(answerPartOne)

Your answer: 4765757080. 
 That's the right answer! You are one gold star closer to decorating the North Pole. [Continue to Part Two]

In [5]:
aocClient.viewPartTwo()

--- Part Two --- 
 The Elves just remembered: they can only switch out tiles that are red or green . So, your rectangle can only include red or green tiles. 
 In your list, every red tile is connected to the red tile before and after it by a straight line of green tiles . The list wraps, so the first red tile is also connected to the last red tile. Tiles that are adjacent in your list will always be on either the same row or the same column. 
 Using the same example as before, the tiles marked X would be green: 
 ..............
.......#XXX#..
.......X...X..
..#XXXX#...X..
..X........X..
..#XXXXXX#.X..
.........X.X..
.........#X#..
..............
 
 In addition, all of the tiles inside this loop of red and green tiles are also green. So, in this example, these are the green tiles: 
 ..............
.......#XXX#..
.......XXXXX..
..#XXXX#XXXX..
..XXXXXXXXXX..
..#XXXXXX#XX..
.........XXX..
.........#X#..
..............
 
 The remaining tiles are never red nor green. 
 The rectangle you choose still must have red tiles in opposite corners, but any other tiles it includes must now be red or green. This significantly limits your options. 
 For example, you could make a rectangle out of red and green tiles with an area of 15 between 7,3 and 11,1 : 
 ..............
.......OOOO O ..
.......OOOOO..
..#XXXX O OOOO..
..XXXXXXXXXX..
..#XXXXXX#XX..
.........XXX..
.........#X#..
..............
 
 Or, you could make a thin rectangle with an area of 3 between 9,7 and 9,5 : 
 ..............
.......#XXX#..
.......XXXXX..
..#XXXX#XXXX..
..XXXXXXXXXX..
..#XXXXXX O XX..
.........OXX..
......... O X#..
..............
 
 The largest rectangle you can make in this example using only red and green tiles has area 24 . One way to do this is between 9,5 and 2,3 : 
 ..............
.......#XXX#..
.......XXXXX..
.. O OOOOOOOXX..
..OOOOOOOOXX..
..OOOOOOO O XX..
.........XXX..
.........#X#..
..............
 
 Using two red tiles as opposite corners, what is the largest area of any rectangle you can make using only red and green tiles?

In [7]:
val tiles = input.trim().lines().map { line ->
    val (x, y) = line.split(",").map { it.trim().toLong() }
    Tile(x, y)
}

// Create edges list to represent the polygon boundary.
// Connect last tile back to first to close the loop.
val edges = tiles.zip(tiles.drop(1) + tiles.first())

fun isRectangleInsidePolygon(t1: Tile, t2: Tile): Boolean {
    val minX = min(t1.x, t2.x)
    val maxX = max(t1.x, t2.x)
    val minY = min(t1.y, t2.y)
    val maxY = max(t1.y, t2.y)

    // 1. Ray Casting Check: Is the center of the rectangle inside the polygon?
    // We use a floating point center to avoid hitting vertices exactly.
    val midX = (minX + maxX) / 2.0
    val midY = (minY + maxY) / 2.0
    var inside = false

    for ((p1, p2) in edges) {
        // Check intersection with vertical edges for ray casting (shooting ray to +X)
        // We ignore horizontal edges for the ray cross check.
        if ((p1.y > midY) != (p2.y > midY)) {
            // Calculate X coordinate where the edge crosses midY
            val intersectX = (p2.x - p1.x) * (midY - p1.y) / (p2.y - p1.y).toDouble() + p1.x
            if (midX < intersectX) {
                inside = !inside
            }
        }
    }

    if (!inside) return false

    // 2. Intersection Check: Does any polygon edge cut THROUGH the rectangle?
    // An edge is allowed to be ON the boundary (green tiles), but not strictly inside.
    for ((p1, p2) in edges) {
        if (p1.x == p2.x) {
            // Vertical Edge
            // Check if X is strictly between rectangle sides
            if (p1.x > minX && p1.x < maxX) {
                // Check if Y-intervals overlap
                val edgeMinY = min(p1.y, p2.y)
                val edgeMaxY = max(p1.y, p2.y)
                if (max(minY, edgeMinY) < min(maxY, edgeMaxY)) return false
            }
        } else {
            // Horizontal Edge
            // Check if Y is strictly between rectangle top/bottom
            if (p1.y > minY && p1.y < maxY) {
                // Check if X-intervals overlap
                val edgeMinX = min(p1.x, p2.x)
                val edgeMaxX = max(p1.x, p2.x)
                if (max(minX, edgeMinX) < min(maxX, edgeMaxX)) return false
            }
        }
    }

    return true
}

val maxArea = tiles.maxOf { t1 ->
    tiles.maxOf { t2 ->
        if (isRectangleInsidePolygon(t1, t2)) {
            (abs(t1.x - t2.x) + 1) * (abs(t1.y - t2.y) + 1)
        } else {
            0L
        }
    }
}

aocClient.submitPartTwo(maxArea)

Your answer: 1498673376. 
 That's the right answer! You are one gold star closer to decorating the North Pole. 
 You have completed Day 9! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .